# Quick Start

Below is a simple demo of interaction with the environment.

In [1]:
from maro.simulator import Env
from maro.simulator.scenarios.citi_bike.common import Action, DecisionEvent

env = Env(scenario="citi_bike", topology="toy.3s_4t", start_tick=0, durations=1440, snapshot_resolution=30)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False

while not is_done:
    action: Action = None
    metrics, decision_event, is_done = env.step(action)

01:57:20 | WARNING | Binary data files for scenario: citi_bike topology: toy.3s_4t not found.
01:57:20 | WARNING | Generating temp binary data file for scenario: citi_bike topology: toy.3s_4t pid: 47618. If you want to keep the data, please use MARO CLI command 'maro env data generate -s citi_bike -t toy.3s_4t' to generate the binary data files first.
01:57:20 | INFO    | Generating trip data for topology toy.3s_4t .
01:57:21 | INFO    | Cleaning weather data
01:57:21 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/1f163cbe71a740ec/trips.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/1f163cbe71a740ec/trips.bin
01:57:29 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/4ab79f60026a460c/weather.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/4ab79f60026a460c/KNYC_daily.bin


# Environment of the bike repositioning

To initialize an environment, you need to specify the values of several parameters:
- **scenario**: The target scenario of this Env. "citi_bike" denotes for the bike repositioning.
- **topology**: The target topology of this Env.
   + There are some predefined topologies in MARO, that you can directly use it as in the demo.
   + Also, you can define your own topologies following the guidance in the [doc](docs/customization/new_topology.rst).       #TODO
- **start_tick**: The start tick of this Env, 1 tick corresponds to 1 minute in citi_bike.
   + In the demo above, *start_tick=0* indicates a simulation start from the beginning of the given topology.
- **durations**: The duration of thie Env, in the unit of tick/minute.
   + In the demo above, *durations=1440* indicates a simulation length of 1 day (24h * 60min/h).
- **snapshot_resolution**: The time granularity of maintaining the snapshots of the environments, in the unit of tick/minute.
   + In the demo above, *snapshot_resolution=30* indicates that a snapshot will be created and saved every 30 minutes during the simulation.

You can get all available scenarios and topologies by calling:

In [2]:
from maro.simulator.utils import get_scenarios, get_topologies
from pprint import pprint
from typing import List

scenarios: List[str] = get_scenarios()
topologies: List[str] = get_topologies(scenario='citi_bike')

pprint(f'The available scenarios in MARO:')
pprint(scenarios)

print()
pprint(f'The predefined topologies in Citi Bike:')             # TODO: update the ordered output
pprint(topologies)

'The available scenarios in MARO:'
['citi_bike', 'ecr']

'The predefined topologies in Citi Bike:'
['ny.201912',
 'ny.201808',
 'ny.201907',
 'ny.202005',
 'ny.201812',
 'ny.201804',
 'toy.3s_4t',
 'toy.4s_4t',
 'ny.201908',
 'ny.201910',
 'train',
 'ny.201909',
 'ny.202002',
 'ny.201811',
 'ny.201906',
 'ny.201802',
 'ny.201803',
 'ny.201905',
 'ny.202003',
 'ny.201805',
 'ny.201809',
 'toy.5s_6t',
 'ny.201801',
 'ny.201904',
 'ny.201902',
 'ny.201901',
 'ny.201911',
 'ny.201903',
 'ny.202001',
 'ny.202004',
 'ny.201806',
 'ny.201807',
 'ny.202006',
 'ny.201810']


Once you created an instance of the environment, you can easily access the real-time information of this environment, like:

In [3]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for citi_bike scenario
env = Env(scenario="citi_bike", topology="toy.3s_4t", start_tick=0, durations=1440, snapshot_resolution=30)

# The current tick
tick: int = env.tick
print(f"The current tick: {tick}.")

# The current frame index, which indicates the index of current frame in the snapshot-list
frame_index: int = env.frame_index
print(f"The current frame index: {frame_index}.")

# The agent index list in the environment
agent_idx_list: List[int] = env.agent_idx_list
print(f"There are {len(agent_idx_list)} agents in this Env.")

# The whole snapshot-list of the environment, snapshots are taken in the granularity of the given snapshot_resolution
# The example of how to use the snapshot will be shown later
snapshot_list: SnapshotList = env.snapshot_list
print(f"There will be {len(snapshot_list)} snapshots in total.")

# The summary info of the environment
summary: dict = env.summary
print(f"\nEnv Summary:")
pprint(summary)

# The metrics of the environment
metrics: dict = env.metrics
print(f"\nEnv Metrics:")
pprint(metrics)

01:57:29 | WARNING | Binary data files for scenario: citi_bike topology: toy.3s_4t not found.
01:57:30 | WARNING | Generating temp binary data file for scenario: citi_bike topology: toy.3s_4t pid: 47618. If you want to keep the data, please use MARO CLI command 'maro env data generate -s citi_bike -t toy.3s_4t' to generate the binary data files first.
01:57:30 | INFO    | Generating trip data for topology toy.3s_4t .
01:57:31 | INFO    | Cleaning weather data
01:57:31 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/ef2f7d085ac24336/trips.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/ef2f7d085ac24336/trips.bin
01:57:38 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/8c5f3ad4fc8b49e9/weather.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/8c5f3ad4fc8b49e9/KNYC_daily.bin
The current tick: 0.
The current frame index: 0.
There are 3 agents in this Env.
There will be 48 snapsho

# Interaction with the environment

Before starting interaction with the environment, we need to know **DecisionEvent** and **Action** first.

## DecisionEvent

Once the environment need the agent's response to promote the simulation, it will throw an **DecisionEvent**. In the scenario of citi_bike, the information of each DecisionEvent is listed as below:
- **station_idx**: the id of the station/agent that needs to respond to the environment
- **tick**: the corresponding tick
- **frame_index**: the corresponding frame index, that is the index of the corresponding snapshot in the snapshot list
- **type**: the decision type of this decision event. In citi_bike scenario, there are 2 types:
   - **Supply**: There is too many bikes in the corresponding station, it's better to reposition some of them to other stations.
   - **Demand**: There is no enough bikes in the corresponding station, it's better to reposition bikes from other stations
- **action_scope**: a dictionary of valid action items.
   - The key of the item indicates the station/agent id;
   - The meaning of the value differs for different decision type:
      - If the decision type is Supply, the value of the station itself means its bike inventory at that moment, while the value of other target stations means the number of their empty docks;
      - If the decision type is Demand, the value of the station itself means the number of its empty docks, while the value of other target stations means their bike inventory.

## Action

Once we get a **DecisionEvent** from the envirionment, we should respond with an **Action**. Valid Action could be:
- None, which means do nothing.
- A valid Action instance, including:
   - **from_station_idx**: int, the id of the source station of the bike transportation
   - **to_station_idx**: int, the id of the destination station of the bike transportation
   - **number**: int, the quantity of the bike transportation

## Generate random actions based on the DecisionEvent

The demo code in the Quick Start part has shown an interaction mode that doing nothing(responding with None action). Here we read the detailed information about the DecisionEvent and generate random actions based on it.

In [4]:
from maro.simulator import Env
from maro.simulator.scenarios.citi_bike.common import Action, DecisionEvent, DecisionType

import random

# Initialize an Env for citi_bike scenario
env = Env(scenario="citi_bike", topology="toy.3s_4t", start_tick=0, durations=1440, snapshot_resolution=30)

metrics: object = None
decision_event: DecisionEvent = None
is_done: bool = False
action: Action = None

# Start the env with a None Action
metrics, decision_event, is_done = env.step(action)

while not is_done:
    if decision_event.type == DecisionType.Supply:
        # the value of the station itself means the bike inventory if Supply
        self_bike_inventory = decision_event.action_scope[decision_event.station_idx]
        # the value of other stations means the quantity of empty docks if Supply
        target_idx_dock_tuple_list = [
            (k, v) for k, v in decision_event.action_scope.items() if k != decision_event.station_idx
        ]
        # random choose a target station weighted by the quantity of empty docks
        target_idx, target_dock = random.choices(
            target_idx_dock_tuple_list,
            weights=[item[1] for item in target_idx_dock_tuple_list]
        )[0]
        # generate the corresponding random Action
        action = Action(
            from_station_idx=decision_event.station_idx,
            to_station_idx=target_idx,
            number=random.randint(0, min(self_bike_inventory, target_dock))
        )

    elif decision_event.type == DecisionType.Demand:
        # the value of the station itself means the quantity of empty docks if Demand
        self_available_dock = decision_event.action_scope[decision_event.station_idx]
        # the value of other stations means their bike inventory if Demand
        target_idx_inventory_tuple_list = [
            (k, v) for k, v in decision_event.action_scope.items() if k != decision_event.station_idx
        ]
        # random choose a target station weighted by the bike inventory
        target_idx, target_inventory = random.choices(
            target_idx_inventory_tuple_list,
            weights=[item[1] for item in target_idx_inventory_tuple_list]
        )[0]
        # generate the corresponding random Action
        action = Action(
            from_station_idx=target_idx,
            to_station_idx=decision_event.station_idx,
            number=random.randint(0, min(self_available_dock, target_inventory))
        )

    else:
        action = None
    
    # Random sampling some records to show in the output   TODO
#     if random.random() > 0.95:
#         print("*************\n{decision_event}\n{action}")
    
    # Respond the environment with the generated Action
    metric, decision_event, is_done = env.step(action)

01:57:39 | WARNING | Binary data files for scenario: citi_bike topology: toy.3s_4t not found.
01:57:39 | WARNING | Generating temp binary data file for scenario: citi_bike topology: toy.3s_4t pid: 47618. If you want to keep the data, please use MARO CLI command 'maro env data generate -s citi_bike -t toy.3s_4t' to generate the binary data files first.
01:57:39 | INFO    | Generating trip data for topology toy.3s_4t .
01:57:40 | INFO    | Cleaning weather data
01:57:40 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/d13e1b3974144696/trips.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/d13e1b3974144696/trips.bin
01:57:48 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/d3015f9c7b824a25/weather.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/d3015f9c7b824a25/KNYC_daily.bin


## Get the environment observation

You can also implement other strategies or build models to take action. At this time, real-time information and historical records of the environment are very important for making good decisions. In this case, the the environment snapshot list is exactly what you need.

The information in the snapshot list is indexed by 3 dimensions:
- A frame index or a frame index list. (int or list of int) Empty indicates for all time slides till now
- A station id (list). (int of list of int) Empty indicates for all stations/agents
- An Attribute name (list). (str of list of str) You can get all available attributes in env.summary as shown before.

The return value from the snapshot list is a numpy.ndarray with shape **(frame * station * attribute, )**.

More detailed introduction to the snapshot list is [here](). # TODO: add hyper-link

In [5]:
from maro.simulator import Env
from pprint import pprint


# Initialize an Env for citi_bike scenario
env = Env(scenario="citi_bike", topology="toy.3s_4t", start_tick=0, durations=1440, snapshot_resolution=30)

# The summary info of the environment
print(f"\nEnv Summary - matrices:")
pprint(env.summary['node_detail']['stations'])

01:57:48 | WARNING | Binary data files for scenario: citi_bike topology: toy.3s_4t not found.
01:57:48 | WARNING | Generating temp binary data file for scenario: citi_bike topology: toy.3s_4t pid: 47618. If you want to keep the data, please use MARO CLI command 'maro env data generate -s citi_bike -t toy.3s_4t' to generate the binary data files first.
01:57:48 | INFO    | Generating trip data for topology toy.3s_4t .
01:57:49 | INFO    | Cleaning weather data
01:57:49 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/54438691a1204cac/trips.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/54438691a1204cac/trips.bin
01:57:57 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/7f8aac5597e248f7/weather.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/7f8aac5597e248f7/KNYC_daily.bin

Env Summary - matrices:
{'attributes': {'bikes': {'slots': 1, 'type': 'i'},
                'capacity': 

In [6]:
from maro.backends.frame import SnapshotList
from maro.simulator import Env
from pprint import pprint
from typing import List


# Initialize an Env for citi_bike scenario, from 07:00 to 12:00
env = Env(scenario="citi_bike", topology="toy.3s_4t", start_tick=420, durations=300, snapshot_resolution=30)

# Run the environment to the end
_, _, is_done = env.step(None)
while not is_done:
    _, _, is_done = env.step(None)

# Get trip requirement from snapshot list by directly using station id and attribute name
station_id = 2
trip_info = env.snapshot_list["stations"][:station_id:"trip_requirement"]
print(type(trip_info), trip_info.shape)
print(f"Trip requirements for station {station_id} with time going by: {trip_info}\n")

# Get capacity and bikes from snapshot list simultaneously by using attribute list
attribute_list = ["capacity", "bikes"]
info = env.snapshot_list["stations"][::attribute_list]
print(type(info), info.shape)

# Reshape the info of capacity and bikes into a user-friendly shape
num_attributes = len(attribute_list)
num_frame = env.frame_index + 1
num_stations = len(env.agent_idx_list)
info = info.reshape(num_frame, num_stations, num_attributes)
print(type(info), info.shape)

# Pring and show the change of bikes in some stations:
bikes_idx = 1
for station_id in env.agent_idx_list:
    print(f"Station {station_id}: {info[:, station_id, bikes_idx]}")

01:57:57 | WARNING | Binary data files for scenario: citi_bike topology: toy.3s_4t not found.
01:57:57 | WARNING | Generating temp binary data file for scenario: citi_bike topology: toy.3s_4t pid: 47618. If you want to keep the data, please use MARO CLI command 'maro env data generate -s citi_bike -t toy.3s_4t' to generate the binary data files first.
01:57:57 | INFO    | Generating trip data for topology toy.3s_4t .
01:57:58 | INFO    | Cleaning weather data
01:57:58 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/af340276c9d34ef2/trips.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/af340276c9d34ef2/trips.bin
01:58:06 | INFO    | Building binary data from /home/Jinyu/.maro/data/citi_bike/.source/.clean/toy.3s_4t/083b42d582f24654/weather.csv to /home/Jinyu/.maro/data/citi_bike/.build/toy.3s_4t/083b42d582f24654/KNYC_daily.bin
<class 'numpy.ndarray'> (10,)
Trip requirements for station 2 with time going by: [15. 15. 15. 15. 15. 15